## Batch Profile Modeling

Using the ESAT simulator to evaluate potential approaches to optimize modeling very large datasets.

The first approach will look at implementing and validating the following workflow:
1. Create a subset dataset of the input by randomly selecting N values from the input/uncertainty.
2. Train a single model on that data until convergence.
3. Use the factor profile H matrix to calculate a W for the complete dataset.
4. Calculate Q(full)
5. Take a new subset of the data, restart training with the prior H.
6. Repeat until Q(full) is no longer decreasing.

Run full dataset model with the same random seed and evaluate the difference in loss and factor profiles.

#### Code Imports

In [2]:
from esat.data.datahandler import DataHandler
from esat.model.batch_sa import BatchSA
from esat.model.sa import SA
from esat.data.analysis import ModelAnalysis, BatchAnalysis
from esat_eval.simulator import Simulator
from esat.estimator import FactorEstimator
from esat_eval.factor_catalog import FactorCatalog, Factor

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import min_weight_full_bipartite_matching

from tqdm.notebook import tqdm

import plotly.graph_objects as go
import pandas as pd
import numpy as np
import copy

#### Synthetic Dataset

Generate a synthetic dataset where the factor profiles and contributions are pre-determined for model output analysis.

In [3]:
# Synethic dataset parameters
seed = 42
syn_factors = 6                # Number of factors in the synthetic dataset
syn_features = 40              # Number of features in the synthetic dataset
syn_samples = 10000             # Number of samples in the synthetic dataset
outliers = True                # Add outliers to the dataset
outlier_p = 0.10               # Decimal percent of outliers in the dataset
outlier_mag = 1.25                # Magnitude of outliers
contribution_max = 2           # Maximum value of the contribution matrix (W) (Randomly sampled from a uniform distribution)
noise_mean_min = 0.03          # Min value for the mean of noise added to the synthetic dataset, used to randomly determine the mean decimal percentage of the noise for each feature.
noise_mean_max = 0.05          # Max value for the mean of noise added to the synthetic dataset, used to randomly determine the mean decimal percentage of the noise for each feature.
noise_scale = 0.1              # Scale of the noise added to the synthetic dataset
uncertainty_mean_min = 0.04    # Min value for the mean uncertainty of a data feature, used to randomly determine the mean decimal percentage for each feature in the uncertainty dataset. 
uncertainty_mean_max = 0.06    # Max value for the mean uncertainty of a data feature, used to randomly determine the mean decimal percentage for each feature in the uncertainty dataset. 
uncertainty_scale = 0.01       # Scale of the uncertainty matrix

In [3]:
# Initialize the simulator with the above parameters
simulator = Simulator(seed=seed,
                      factors_n=syn_factors,
                      features_n=syn_features,
                      samples_n=syn_samples,
                      outliers=outliers,
                      outlier_p=outlier_p,
                      outlier_mag=outlier_mag,
                      contribution_max=contribution_max,
                      noise_mean_min=noise_mean_min,
                      noise_mean_max=noise_mean_max,
                      noise_scale=noise_scale,
                      uncertainty_mean_min=uncertainty_mean_min,
                      uncertainty_mean_max=uncertainty_mean_max,
                      uncertainty_scale=uncertainty_scale
                     )

21-Apr-25 16:07:12 - Synthetic profiles generated


In [4]:
# Example command for passing in a custom factor profile matrix, instead of the randomly generated profile matrix.
# my_profile = np.ones(shape=(syn_factors, syn_features))
# simulator.generate_profiles(profiles=my_profile)

In [5]:
# Example of how to customize the factor contributions. Curve_type options: 'uniform', 'decreasing', 'increasing', 'logistic', 'periodic'
# simulator.update_contribution(factor_i=0, curve_type="logistic", scale=0.1, frequency=0.5)
# simulator.update_contribution(factor_i=1, curve_type="periodic", minimum=0.0, maximum=1.0, frequency=0.5, scale=0.1)
# simulator.update_contribution(factor_i=2, curve_type="increasing", minimum=0.0, maximum=1.0, scale=0.1)
# simulator.update_contribution(factor_i=3, curve_type="decreasing", minimum=0.0, maximum=1.0, scale=0.1)
# simulator.plot_synthetic_contributions()

#### Load Data
Assign the processed data and uncertainty datasets to the variables V and U. These steps will be simplified/streamlined in a future version of the code.

In [5]:
syn_input_df, syn_uncertainty_df = simulator.get_data()

21-Apr-25 16:07:13 - Synthetic data generated
21-Apr-25 16:07:14 - Synthetic uncertainty data generated
21-Apr-25 16:07:14 - Synthetic dataframes completed
21-Apr-25 16:07:14 - Synthetic source apportionment instance created.


In [6]:
data_handler = DataHandler.load_dataframe(input_df=syn_input_df, uncertainty_df=syn_uncertainty_df)
V, U = data_handler.get_data()

#### Input Parameters

In [7]:
index_col = "Date"                  # the index of the input/uncertainty datasets
# factors = syn_factors               # the number of factors
factors = 6
method = "ls-nmf"                   # "ls-nmf", "ws-nmf"
models = 20                         # the number of models to train
init_method = "col_means"           # default is column means "col_means", "kmeans", "cmeans"
init_norm = True                    # if init_method=kmeans or cmeans, normalize the data prior to clustering.
seed = 42                           # random seed for initialization
max_iterations = 20000              # the maximum number of iterations for fitting a model
converge_delta = 0.1                # convergence criteria for the change in loss, Q
converge_n = 25                     # convergence criteria for the number of steps where the loss changes by less than converge_delta
verbose = True                      # adds more verbosity to the algorithm workflow on execution.
optimized = True                    # use the Rust code if possible
parallel = True                     # execute the model training in parallel, multiple models at the same time

### Train Batch Profile

In [ ]:
def calculate_W(V, U, H):
    H[H <= 0.0] = 1e-8
    W = np.matmul(V * np.divide(1, U), H.T)
    return W

def q_loss(V, U, H, W):
    residuals = (V-np.matmul(W, H))/U
    return np.sum(residuals)

def mse(V, U, H, W):
    WH = np.matmul(W, H)
    residuals = ((V-WH)/U)**2
    return np.sum(residuals)/V.size

def compare_H(H1, H2):
    correlation_matrix = np.zeros((H1.shape[0], H2.shape[0]))
    for i in range(H1.shape[0]):
        f1 = H1[i].astype(float)
        for j in range(H2.shape[0]):
            f2 = H2[j].astype(float)
            corr_matrix = np.corrcoef(f2, f1)
            corr = corr_matrix[0, 1]
            r_sq = corr ** 2
            correlation_matrix[i,j] = r_sq
    return correlation_matrix
            
def plot_correlations(matrix):
    header = [f"Factor {i}" for i in range(matrix.shape[0])]
    fig = go.Figure(data=[go.Table(header=dict(values=header), cells=dict(values=matrix))])
    fig.show()

def prepare_data(V, U, i_selection):
    _V = pd.DataFrame(V.copy()[i_selection,:])
    _U = pd.DataFrame(U.copy()[i_selection,:])
    
    for f in _V.columns:
        _V[f] = pd.to_numeric(_V[f])
        _U[f] = pd.to_numeric(_U[f])
    return _V.to_numpy(), _U.to_numpy()

### Version 2 of the FactorCatalog

FactorCatalog V2 takes a more robust approach to grouping factor profiles from multiple models with the grouping occuring after all factor profiles have been collected. The updated procedure
is designed to be used to investigate potential solutions for creating models for very large datasets using subsets of the data. The algorithm is described as:
1. Specify your hyper-parameters: samples_n, batches_n, models_n, random_seed, correlation_threshold, factors_k
2. For each batch in batches_n.
3. Create a subset dataset using samples_n randomly selected values from V/U.
4. Created a batchSA instance of models_n, using random_seed and factors_k.
5. Add each output factor to the FactorCatalog (factor model_i, fit_rmse, factor_i, H)
6. Once all batches are completed, cluster the factor collection using a constrained k-means cluster function.
7. Score the models based upon a heuristic, such as the sum of (cluster_cor_avg*cluster_members)
8. Evaluate the clustered profile matrix, using the cluster centroid values, using the complete dataset.

The primary modification of the FactorCatalog is to use the constrained k-means clustering function for grouping 'like' factor profiles. The procedure will work by:
1. Starting with the factors_k=clusters_n, calculate the correlation of all factors/points to the centroids, by model.
2. Initialize the clusters by randomly creating clusters or by selecting clusters_n 'dissimilar' factors.
3. Randomly shuffle the model order for factor assignment:
   1. Assign the factors to the clusters by order of correlation, closer to 1.0 goes first.
   2. If more than one factor in a model would be assigned to the same cluster assign the factor with the highest cor to that cluster and then repeat excluding that cluster(s) until all factors are assigned.
   3. If any given factor does not have a correlation above the specified threshold, create a new cluster centered at that point.
4. At the end of each assignment iteration, remove any cluster which has no members.
5. Once max_assignment_n iterations is reached stop, or when a reassignment doesn't change.

The constrained k-means clustered is a standard clustering approach with the exception that the distance is calculated as 1/r2 of the point to the cluster centroid. Two other differences are a) the number of clusters can increase and decrease depending on correlation threshold and by the constraint that a model can only contribute one factor to any given cluster.

Once all the factors for all models have been clustered, the FactorCatalog models can be scored based upon the heuristic stated in stage A7. The best model, or any selected model, factor profile matrix (H) can then be selected for final evaluation. The factor profile H is not what was produced by the model, but is the mean values of the FactorCatalog's factor (the centroid of the cluster that those factors were assigned to). This approach allows for the factor profile values to be provided as a distributed of possible values for each feature, or demonstrating potential uncertainty in the factor profile. The clustered factor profile is then used to fit the full dataset, but keeping H constant. The loss can then be evaluated against what is calculated for a long-running brute force approach. 

An evaluation of the impact on the model/W matrix and loss given a random selection, MC, simulation of the factor profile would be an interesting next step.

In [ ]:
class Factor:
    def __init__(self,
                 factor_id,
                 profile,
                 model_id
                ):
        self.factor_id = factor_id
        self.profile = profile
        self.model_id = model_id
        self.cluster_id = None
        self.cor = None

    def assign(self, cluster_id, cor):
        self.cluster_id = cluster_id
        self.cor = cor

    def distance(self, cluster):
        f1 = np.array(self.profile).astype(float)
        f2 = np.array(cluster).astype(float)
        corr_matrix = np.corrcoef(f2, f1)
        corr = corr_matrix[0, 1]
        r_sq = corr ** 2
        return r_sq


class Model:
    def __init__(self,
                 model_id):
        self.model_id = model_id
        self.factors = []

        self.score = None
        
    def add_factor(self, factor):
        self.factors.append(factor)


class Cluster:
    def __init__(self,
                 centroid: np.ndarray
                ):
        self.centroid = centroid
        self.factors = []
        self.count = 0

        self.mean_r2 = 0
        self.std = 0
        self.min_values = np.full(len(centroid), np.nan)
        self.max_values = np.full(len(centroid), np.nan)

    def __len__(self):
        return self.count

    def add(self, factor: Factor):
        self.factors.append(factor)
        self.count += 1
        self.min_values = np.minimum(self.min_values, factor)
        self.max_values = np.maximum(self.max_values, factor)
        self.mean_r2 = np.mean([factor.cor for factor in self.factors])
        self.std = np.std([factor.profile for factor in self.factors], axis=0)

    def purge(self):
        self.factors = []
        self.count = 0
        self.mean_r2 = 0
        self.std = 0
        self.min_values = np.full(len(centroid), np.nan)
        self.max_values = np.full(len(centroid), np.nan)

    def recalculate(self):
        profile_list = np.array([factor.profile for factor in self.factors])
        new_centroid = np.mean(profile_list, axis=0)
        self.centroid = new_centroid


class BatchFactorCatalog:
    def __init__(self
                 n_factors: int,
                 threshold: float = 0.8,
                 seed: int = 42
                ):
        self.n_factors = n_factors
        self.threshold = threshold

        self.rng = np.random.default_rng(seed)

        self.models = {}
        self.model_count = 0
        self.factor_collection = {}
        self.factor_count = 0

        # Min and max values for all factor vectors, used for random initialization of the centroids in clustering
        self.factor_min = None
        self.factor_max = None

        self.clusters = []

    def add_model(self, model: SA, norm: bool = True):
        model_id = self.model_count
        model_factor_ids = []
        norm_H = model.H / np.sum(model.H, axis=0)
        model = Model(model_id=model_id)
        for i in range(model.H.shape[0]):
            factor_id = self.factor_count
            self.factor_count += 1
            model_factor_ids.append(factor_id)
            i_H = norm_H if norm else model.H 
            factor = Factor(factor_id=factor_id, profile=i_H[i], model_id=model_id)
            
            model.add_factor(factor)
            self.factor_collection[str(factor_id)] = factor
            self.update_ranges(i_H[i])
            
        self.models[str(model_id)] = model
        self.model_count += 1

    def score(self):
        pass

    def update_ranges(self, factor):
        if self.factor_min is None and self.factor_max is None:
            self.factor_min = copy.copy(factor.profile)
            self.factor_max = copy.copy(factor.profile)
        else:
            self.factor_min = np.minimum(self.factor_min, factor.profile)
            self.factor_max = np.maximum(self.factor_max, factor.profile)

    def initialize_clusters(self):
        for k in range(self.n_factors):
            new_centroid = np.zeros(len(self.factor_min))
            for i in range(len(self.factor_min)):
                i_v = self.rng.uniform(low=self.factor_min[i], high=self.factor_max[i])
                new_centroid[i] = i_v
            cluster = Cluster(centroid=new_centroid)
            self.clusters.append(cluster)

    def purge_clusters(self):
        for cluster in self.clusters:
            cluster.purge()

    def distance(self, factor1, factor2):
        f1 = np.array(factor1).astype(float)
        f2 = np.array(factor2).astype(float)
        corr_matrix = np.corrcoef(f2, f1)
        corr = corr_matrix[0, 1]
        r_sq = corr ** 2
        return r_sq

    def calculate_centroids(self, cluster):
        new_centroid = list(np.mean(np.array(cluster), axis=0))
        return new_centroid

    def cluster_cleanup(self):
        drop_clusters = []
        for i in range(len(self.centroids)):
            centroid_i = self.centroids[i]
            for j in range(i, len(self.centroids)):
                centroid_j = centroids[j]
                ij_cor = self.distance(centroid_i.centroid centroid_j.centroid)
                if ij > self.threshold:
                    smaller_cluster = i if len(clusters[i]) < len(clusters[j]) else j
                    if smaller_cluster not in drop_clusters:
                        drop_clusters.append(smaller_cluster)
        new_clusters = []
        for i in range(len(self.centroids)):
            if i not in drop_clusters:
                new_clusters.append(clusters[i])
        self.clusters = new_clusters             

    def run(self, max_iterations: int = 200):
        self.initialize_clusters()
        # The initial number of clusters is equal to n_factors
        n_clusters = self.n_factors
        
        converged = False
        current_iter = 0
        while not converged:
            if current_iter >= max_iterations:
                logger.info(f"Factor clustering did not converge after {max_iterations} iterations.")
                break
            self.purge_clusters()

            model_list = self.rng.permutation(self.models.keys())
            for model_i in model_list:
                model_factors = self.models[model_i].factors
                factor_dist = {}
                factor_hi = {}
                # Calculate distances for all factors in the model to all centroids and then order the distances.
                for factor_i in model_factors:
                    distances = [(j, self.distance(self.factor_collection[str(factor_i)][1], centroid)) for j, centroid in enumerate(centroids)]
                    distances.sorted(key=lambda x: x[1], reverse=True)
                    factor_dist[str(factor_i)] = distances
                    factor_hi[str(factor_i)] = distances[0]
                already_assigned = []
                factor_hi = dict(sorted(x.items(), key=lambda x: x[1], reverse=True))
                # Assign factors to clusters, if model hasn't contributed to the cluster already and if the correlation is above the threshold
                for factor_id in factor_hi.keys():
                    # iterate through list of clusters until the first one that hasn't already been assigned.
                    cluster_idx = -1
                    for cluster_i, correlation_i in factor_dist[factor_id].items():
                        if cluster_i not in already_assigned and correlation_i >= self.threshold:
                            cluster_idx = cluster_i
                            break
                    if cluster_idx != -1:
                        clusters[cluster_idx].append(factor_id)
                        already_assigned.append(cluster_idx)
                    else:
                        n_clusters += 1
                        clusters.append([factor_id])
                        already_assigned.append(len(clusters))

                # Recalculate centroids of clusters
                new_centroids = [self.calculate_centroids([factor_collection[str(factor_i)][1] for factor_i in cluster]) for cluster in clusters]
                new_centroids, clusters =  self.cluster_cleanup(new_centroids, clusters)
                if new_centroids == centroids:
                    converge = True
                else:
                    centroids = new_centroids
                current_iter += 1
                    
        self.centroids = centroids
        self.clusters = clusters

In [ ]:
%%time

rng = np.random.default_rng(seed)
batch_size = 1000
max_batches = 10
i_batches = 0

best_mse = float('inf')
best_model = 0

i_H = None
i_selection = rng.choice(syn_samples, size=batch_size, replace=False, shuffle=True)
i_V, i_U = prepare_data(V=V, U=U, i_selection=i_selection)

factor_catalog = BatchFactorCatalog(n_factors=factors, threshold=0.8, seed=42)

change_p = 0.1
with tqdm(range(max_batches*2), desc="Generating subset profiles. ") as pbar:
    for i in range(max_batches):
        if i > 0:
            j_selection = rng.choice(syn_samples, size=int(batch_size*change_p), replace=False, shuffle=True)
            idx_change = rng.choice(batch_size, size=int(batch_size*change_p), replace=False, shuffle=True)
            i_selection[idx_change] = j_selection
            i_V, i_U = prepare_data(V=V, U=U, i_selection=i_selection)

        batch_sa = BatchSA(V=train_V, U=train_U, H=batch_H, factors=k, models=n_models, method=method, seed=initialization_seed, max_iter=max_iter,
                            converge_delta=converge_delta, converge_n=converge_n, verbose=False)
        _ = sa_models_b.train()
        pbar.update(1)

        factor_catalog.add_model(model=sa, norm=True)
        
        pbar.set_description(f"Generating subset profiles.")


In [ ]:
final_sa = SA(factors=factors, method=method, V=V, U=U, seed=seed, verbose=True)
final_sa.initialize(H=i_H, W=None)
run = final_sa.train(max_iter=2000, converge_delta=converge_delta, converge_n=converge_n)

Generating common profile. MSE: NA:   0%|          | 0/20 [00:00<?, ?it/s]

In [11]:
# m_bi_matrix = csr_matrix(correlation)
# model_mapping = list(min_weight_full_bipartite_matching(m_bi_matrix, maximize=True))
# model_mapping

In [12]:
# cor_values = correlation[model_mapping[0], model_mapping[1]]
# cor_values

In [ ]:
fig = go.Figure(data=go.Scatter(x=list(range(len(batch_mse))), y=batch_mse, mode='lines'))
fig.update_layout(
    title="Aggregated MSE over batches",
    xaxis_title="Batches",
    yaxis_title="MSE",
    width=1200,
    height=800
)
fig.show()

#### Train Full Model

In [13]:
%%time
# Training multiple models, optional parameters are commented out.
sa_models = BatchSA(V=V, U=U, factors=factors, models=models, method=method, seed=seed, max_iter=max_iterations,
                    init_method=init_method, init_norm=init_norm,
                    converge_delta=converge_delta, converge_n=converge_n, 
                    parallel=parallel,
                    verbose=True
                   )
_ = sa_models.train()

08-Apr-25 15:50:34 - Batch Source Apportionment Instance Configuration
08-Apr-25 15:50:34 - -------------------------------------------------
08-Apr-25 15:50:34 - Factors: 6, Method: ls-nmf, Models: 20
08-Apr-25 15:50:34 - Max Iterations: 20000, Converge Delta: 0.1, Converge N: 25
08-Apr-25 15:50:34 - Random Seed: 42, Init Method: col_means
08-Apr-25 15:50:34 - Parallel: True, Verbose: True
08-Apr-25 15:50:34 - -------------------------------------------------
08-Apr-25 15:50:34 - Estimated memory available: 103.7799 Gb
08-Apr-25 15:50:34 - Estimated memory per model: 183.4595 MB
08-Apr-25 15:50:34 - Estimated maximum number of cores: 16
08-Apr-25 15:50:34 - Using 12 cores for parallel processing.
08-Apr-25 15:50:34 - -------------------------------------------------
08-Apr-25 15:50:34 - Running batch SA models in parallel using 12 cores.
08-Apr-25 16:35:57 - Model: 1, Q(true): 1931058.125, MSE(true): 4.827600002288818, Q(robust): 1402891.375, MSE(robust): 3.507200002670288, Seed: 8925

CPU times: total: 3.06 s
Wall time: 45min 23s


#### Batch Analysis

These methods allow for plotting and reviewing of the overall results of the collection of models produced by the BatchSA training.

In [42]:
sa_models.results[0].H[0].shape

(40,)

In [ ]:
# Perform batch model analysis
batch_analysis = BatchAnalysis(batch_sa=sa_models, data_handler=data_handler)
# Plot the loss of the models over iterations
batch_analysis.plot_loss()

In [ ]:
# Plot the loss distribution for the batch models
batch_analysis.plot_loss_distribution()

In [ ]:
# Plot the temporal residuals for each model, the loss by sample, for a specified feature
batch_analysis.plot_temporal_residuals(feature_idx=0)

### Compare to Synthetic Data

Compare the set of batch models to the original synthetic factor data.


In [ ]:
simulator.compare(batch_sa=sa_models)


In [ ]:
simulator.plot_comparison()

In [ ]:
# Get best mapping of the most optimal model (by loss), plot those mapping results
# simulator.plot_comparison(model_i=sa_models.best_model)

In [ ]:
# Save the Simulator instance, saves the instance as a pickle file and saves the synthetic profiles, contributions, data and uncertainty as csv files.
# sim_name = "synthetic"
# sim_output_dir = "D:/git/esat/notebooks/"
# simulator.save(sim_name=sim_name, output_directory=sim_output_dir)

In [ ]:
# Load a previously saved Simulator instance
# simulator_file = "D:/git/esat/notebooks/esat_simulator.pkl"
# simulator_2 = Simulator.load(file_path=simulator_file)
# simulator_2.factor_compare.print_results()

In [ ]:
# Selet the highest correlated model
best_model = simulator.factor_compare.best_model
sa_model = sa_models.results[best_model]
best_model

In [ ]:
# Initialize the Model Analysis module
model_analysis = ModelAnalysis(datahandler=data_handler, model=sa_model, selected_model=best_model)

In [ ]:
# Residual Analysis shows the scaled residual histogram, along with metrics and distribution curves. The abs_threshold parameter specifies the condition for the returned values of the function call as those residuals which exceed the absolute value of that threshold.
abs_threshold = 3.0
threshold_residuals = model_analysis.plot_residual_histogram(feature_idx=5, abs_threshold=abs_threshold)

In [ ]:
print(f"List of Absolute Scaled Residual Greather than: {abs_threshold}. Count: {threshold_residuals.shape[0]}")
threshold_residuals

In [ ]:
# The model output statistics for the estimated V, including SE: Standard Error metrics, and 3 normal distribution tests of the residuals (KS Normal is used in PMF5)
model_analysis.calculate_statistics()
model_analysis.statistics

In [ ]:
# Model feature observed vs predicted plot with regression and one-to-one lines. Feature/Column specified by index.
model_analysis.plot_estimated_observed(feature_idx=2)

In [ ]:
# Model feature timeseries analysis plot showing the observed vs predicted values of the feature, along with the residuals shown below. Feature/column specified by index.
model_analysis.plot_estimated_timeseries(feature_idx=1)

In [ ]:
# Factor profile plot showing the factor sum of concentrations by feature (blue bars), the percentage of the feature as the red dot, and in the bottom plot the normalized contributions by date (values are resampled at a daily timestep for timeseries consistency).
# Factor specified by index.
model_analysis.plot_factor_profile(factor_idx=1)

In [ ]:
# Model factor fingerprint specifies the feature percentage of each factor.
model_analysis.plot_factor_fingerprints(grouped=False)

In [ ]:
# Factor G-Space plot shows the normalized contributions of one factor vs another factor. Factor specified by index.
model_analysis.plot_g_space(factor_1=2, factor_2=1)

In [ ]:
# Factor contribution pie chart shows the percentage of factor contributions for the specified feature, and the corresponding normalized contribution of each factor for that feature (bottom plot). Feature specified by index.
model_analysis.plot_factor_contributions(feature_idx=1)